# Setup

In [1]:
library(tidyverse)
library(lubridate)
library(caret)

── Attaching core tidyverse packages ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lif

In [2]:
df <- read.csv('data_raw/train.csv', na.strings=c("","NA"))

df %>% head
dim(df)

,X,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67 kmpl,1582 CC,126.2 bhp,5,NA,12.50
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5,8.61 Lakh,4.50
3,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77 kmpl,1248 CC,88.76 bhp,7,NA,6.00
4,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.2 kmpl,1968 CC,140.8 bhp,5,NA,17.74
5,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08 kmpl,1461 CC,63.1 bhp,5,NA,3.50
6,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36 kmpl,2755 CC,171.5 bhp,8,21 Lakh,17.50


[1] 5847   14

## Handling Different Units (km/kg and km/L)

There's only 5 rows which have a Mileage in some unit other than kmpl. Hence, we can safely remove the associated rows, as they likely do not measure the same thing.

In [3]:
df[endsWith(df$Mileage, ("km/kg")),]

# filter out all rows that end with km/kg
df <- df %>% filter(!endsWith(df$Mileage, ("km/kg")))

,X,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>,<dbl>
2,2,Honda Jazz V,Chennai,2011,46000,Petrol,Manual,First,13 km/kg,1199 CC,88.7 bhp,5,8.61 Lakh,4.50
25,26,Nissan Micra Diesel XV,Hyderabad,2012,54000,Diesel,Manual,First,16.48 km/kg,1461 CC,63.1 bhp,5,NA,4.25
57,58,Nissan X-Trail SLX AT,Hyderabad,2010,121812,Diesel,Automatic,First,9.49 km/kg,1995 CC,147.6 bhp,5,NA,7.75
NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
NA.1,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA


# Part b: Removing Units

Now we can remove units from Mileage, Engine, Power, and New_Price

In [4]:
# mutate each column by replacing the unit with an empty string, then cast to numeric
df <- df %>% mutate(Mileage = Mileage %>% gsub("kmpl", "", .) %>% as.numeric)
df <- df %>% mutate(Power = Power %>% gsub("bhp", "", .) %>% as.numeric)
df <- df %>% mutate(Engine = Engine %>% gsub("CC", "", .) %>% as.numeric)
df <- df %>% mutate(New_Price = New_Price %>% gsub("Lakh", "", .) %>% as.numeric %>% suppressWarnings)

df %>% head

,X,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,Diesel,Manual,First,19.67,1582,126.20,5,NA,12.50
2,3,Maruti Ertiga VDI,Chennai,2012,87000,Diesel,Manual,First,20.77,1248,88.76,7,NA,6.00
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Diesel,Automatic,Second,15.20,1968,140.80,5,NA,17.74
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,Diesel,Manual,First,23.08,1461,63.10,5,NA,3.50
5,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,Diesel,Automatic,First,11.36,2755,171.50,8,21,17.50
6,8,Volkswagen Vento Diesel Comfortline,Pune,2013,64430,Diesel,Manual,First,20.54,1598,103.60,5,NA,5.20


In [5]:
write.csv(df, 'data_clean/00_removed_units.csv')

# Part a: Missing Values

First, we can find the amount of missing values in each column and the basic statistics of each. We can see that the only columns with missing values is Seats (38), Engine (36), Power (36), and New_Price (5,046)

In [6]:
# calculate the number of NA values for each feature
apply(df, 2, function(x) sum(is.na(x)))

summary(df)

X              Name          Location              Year 
                0                 0                 0                 0 
Kilometers_Driven         Fuel_Type      Transmission        Owner_Type 
                0                 0                 0                 0 
          Mileage            Engine             Power             Seats 
                0                36                36                38 
        New_Price             Price 
             5046                 0

       X            Name             Location              Year     
 Min.   :   1   Length:5842        Length:5842        Min.   :1998  
 1st Qu.:1511   Class :character   Class :character   1st Qu.:2012  
 Median :3016   Mode  :character   Mode  :character   Median :2014  
 Mean   :3014                                         Mean   :2013  
 3rd Qu.:4518                                         3rd Qu.:2016  
 Max.   :6018                                         Max.   :2019  
                                                                    
 Kilometers_Driven  Fuel_Type         Transmission        Owner_Type       
 Min.   :    171   Length:5842        Length:5842        Length:5842       
 1st Qu.:  33431   Class :character   Class :character   Class :character  
 Median :  52588   Mode  :character   Mode  :character   Mode  :character  
 Mean   :  58406                                                           
 3rd Qu.:  72493                                                    

## Impute or Drop

Starting off by looking at the New_Price column, it is obvious that our only option is to impute the data, as removing 5,046 rows from the dataset would be undesired. I'll impute this column by mean.

In [7]:
df$New_Price[is.na(df$New_Price)] <- mean(df$New_Price, na.rm = TRUE)

summary(df$New_Price)

   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
   3.91   20.92   20.92   20.92   20.92   99.92 

Next, let's take a look at the remaining 3 features. We can see that there is a large amount of overlap between the missing values for Engine, Power and Seats (in fact, there is 100% overlap between the NA rows). Additionally, we can see that the New_Price feature for each is not unique, rather, it is the value of the mean that was used in the previous impute step; hence, the values of the New_Price column do not provide any additional variance.

Additionally, we can see that these rows make up only ~0.65% of the dataset.

In [8]:
df[!complete.cases(df),]

cat(sprintf("TOTAL PERCENT OF DATAFRAME: %s", (nrow(df[!complete.cases(df),])) / (nrow(df)) * 100))

,X,Name,Location,Year,Kilometers_Driven,Fuel_Type,Transmission,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
184,194,Honda City 1.5 GXI,Ahmedabad,2007,60006,Petrol,Manual,First,0.00,NA,NA,NA,20.91683,2.95
198,208,Maruti Swift 1.3 VXi,Kolkata,2010,42001,Petrol,Manual,First,16.10,NA,NA,NA,20.91683,2.11
218,229,Ford Figo Diesel,Bangalore,2015,70436,Diesel,Manual,First,0.00,1498,99,NA,20.91683,3.60
707,733,Maruti Swift 1.3 VXi,Chennai,2006,97800,Petrol,Manual,Third,16.10,NA,NA,NA,20.91683,1.75
721,749,Land Rover Range Rover 3.0 D,Mumbai,2008,55001,Diesel,Automatic,Second,0.00,NA,NA,NA,20.91683,26.50
1251,1294,Honda City 1.3 DX,Delhi,2009,55005,Petrol,Manual,First,12.80,NA,NA,NA,20.91683,3.20
1282,1327,Maruti Swift 1.3 ZXI,Hyderabad,2015,50295,Petrol,Manual,First,16.10,NA,NA,NA,20.91683,5.80
1337,1385,Honda City 1.5 GXI,Pune,2004,115000,Petrol,Manual,Second,0.00,NA,NA,NA,20.91683,1.50
1410,1460,Land Rover Range Rover Sport 2005 2012 Sport,Coimbatore,2008,69078,Petrol,Manual,First,0.00,NA,NA,NA,20.91683,40.88


TOTAL PERCENT OF DATAFRAME: 0.650462170489558

As a result of being a low percentage of the overall dataset, and contributing little variance to several of the features, I'll just remove these rows.

In [9]:
df <- df[complete.cases(df),]

apply(df, 2, function(x) sum(is.na(x)))

X              Name          Location              Year 
                0                 0                 0                 0 
Kilometers_Driven         Fuel_Type      Transmission        Owner_Type 
                0                 0                 0                 0 
          Mileage            Engine             Power             Seats 
                0                 0                 0                 0 
        New_Price             Price 
                0                 0

In [10]:
write.csv(df, 'data_clean/01_removed_na.csv')

# Part c: One-Hot Encoding

Both Fuel_Type and Transmission are categorical variables that have two unique values. In order to one-hot encode these features, 4 new columns will be made: Fuel_TypeDiesel, Fuel_TypePetrol, TransmissionAutomatic, and TransmissionManual. Each row in these columns will contain either 1 for presence of that feature, or 0 for absence. The caret library was used for this section.

In [11]:
unique(df$Fuel_Type)
unique(df$Transmission)

# create dummy vars (split features and assign values of 0 or 1)
dummy <- dummyVars(~ Fuel_Type + Transmission, data=df)
df_new <- data.frame(predict(dummy, newdata=df))

df <- df %>% select(-6:-7) %>% cbind(df_new)
df %>% head

[1] "Diesel" "Petrol"

[1] "Manual"    "Automatic"

,X,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_TypeDiesel,Fuel_TypePetrol,TransmissionAutomatic,TransmissionManual
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582,126.20,5,20.91683,12.50,1,0,0,1
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248,88.76,7,20.91683,6.00,1,0,0,1
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968,140.80,5,20.91683,17.74,1,0,1,0
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461,63.10,5,20.91683,3.50,1,0,0,1
5,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,First,11.36,2755,171.50,8,21.00000,17.50,1,0,1,0
6,8,Volkswagen Vento Diesel Comfortline,Pune,2013,64430,First,20.54,1598,103.60,5,20.91683,5.20,1,0,0,1


In [12]:
write.csv(df, 'data_clean/02_one_hot.csv')

# Part d: Adding Features

In this section, two new columns will be added which describes the age of the car, and the average value loss per year. 

In [13]:
df <- df %>%
    mutate(Age = (2024 - Year)) %>%
    mutate(Annual_Value_Loss = ifelse(Age == 0, NA, (New_Price - Price) / Age))

df %>% head

,X,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_TypeDiesel,Fuel_TypePetrol,TransmissionAutomatic,TransmissionManual,Age,Annual_Value_Loss
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582,126.20,5,20.91683,12.50,1,0,0,1,9,0.9352038
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248,88.76,7,20.91683,6.00,1,0,0,1,12,1.2430695
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968,140.80,5,20.91683,17.74,1,0,1,0,11,0.2888031
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461,63.10,5,20.91683,3.50,1,0,0,1,11,1.5833486
5,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,First,11.36,2755,171.50,8,21.00000,17.50,1,0,1,0,8,0.4375000
6,8,Volkswagen Vento Diesel Comfortline,Pune,2013,64430,First,20.54,1598,103.60,5,20.91683,5.20,1,0,0,1,11,1.4288031


In [14]:
write.csv(df, 'data_clean/03_added_features.csv')

# Part e: Common Operations

Here, the following functions will be used:
- select
- filter
- rename
- mutate
- arrange
- summarize
- group_by

## select

This will show the rows for the Engine and Power columns.

In [15]:
df %>% select(c('Engine', 'Power')) %>% head

,Engine,Power
,<dbl>,<dbl>
1,1582,126.20
2,1248,88.76
3,1968,140.80
4,1461,63.10
5,2755,171.50
6,1598,103.60


## filter

This shows all cars that are made after 2015 and have less than 40,000 kilometers driven.

In [16]:
df %>% filter(Year >= 2015 & Kilometers_Driven < 40000) %>% head

,X,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_TypeDiesel,Fuel_TypePetrol,TransmissionAutomatic,TransmissionManual,Age,Annual_Value_Loss
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,First,11.36,2755,171.50,8,21.00000,17.50,1,0,1,0,8,0.43750000
2,10,Maruti Ciaz Zeta,Kochi,2018,25692,First,21.56,1462,103.25,5,10.65000,9.95,0,1,0,1,6,0.11666667
3,17,Maruti Swift DDiS VDI,Jaipur,2017,25000,First,28.40,1248,74.00,5,20.91683,5.99,1,0,0,1,7,2.13240488
4,28,Honda WRV i-VTEC VX,Kochi,2018,37430,First,17.50,1199,88.70,5,10.57000,9.90,0,1,0,1,6,0.11166667
5,42,Mahindra KUV 100 mFALCON D75 K8 Dual Tone,Coimbatore,2017,22033,First,25.32,1198,77.00,6,20.91683,5.58,1,0,0,1,7,2.19097631
6,50,Audi A4 2.0 TDI 177 Bhp Premium Plus,Kochi,2015,13648,First,17.11,1968,174.33,5,20.91683,21.43,1,0,1,0,9,-0.05701843


## rename

Here, we'll rename the Fuel_Type and Transmission one-hot encoded features created earlier by adding an underscore before the classification.

In [17]:
lookup <- c(Fuel_Type_Diesel = "Fuel_TypeDiesel",
            Fuel_Type_Petrol = "Fuel_TypePetrol",
            Transmission_Automatic = "TransmissionAutomatic",
            Transmission_Manual = "TransmissionManual")

df %>% rename(all_of(lookup)) %>% head

,X,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_Type_Diesel,Fuel_Type_Petrol,Transmission_Automatic,Transmission_Manual,Age,Annual_Value_Loss
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582,126.20,5,20.91683,12.50,1,0,0,1,9,0.9352038
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248,88.76,7,20.91683,6.00,1,0,0,1,12,1.2430695
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968,140.80,5,20.91683,17.74,1,0,1,0,11,0.2888031
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461,63.10,5,20.91683,3.50,1,0,0,1,11,1.5833486
5,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,First,11.36,2755,171.50,8,21.00000,17.50,1,0,1,0,8,0.4375000
6,8,Volkswagen Vento Diesel Comfortline,Pune,2013,64430,First,20.54,1598,103.60,5,20.91683,5.20,1,0,0,1,11,1.4288031


## mutate

This section adds a new feature Miles_Driven, based on the Kilometers_Driven feature times a conversion factor.

In [18]:
df %>% mutate(Miles_Driven = Kilometers_Driven * 0.6213711922) %>% head

,X,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_TypeDiesel,Fuel_TypePetrol,TransmissionAutomatic,TransmissionManual,Age,Annual_Value_Loss,Miles_Driven
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1,Hyundai Creta 1.6 CRDi SX Option,Pune,2015,41000,First,19.67,1582,126.20,5,20.91683,12.50,1,0,0,1,9,0.9352038,25476.22
2,3,Maruti Ertiga VDI,Chennai,2012,87000,First,20.77,1248,88.76,7,20.91683,6.00,1,0,0,1,12,1.2430695,54059.29
3,4,Audi A4 New 2.0 TDI Multitronic,Coimbatore,2013,40670,Second,15.20,1968,140.80,5,20.91683,17.74,1,0,1,0,11,0.2888031,25271.17
4,6,Nissan Micra Diesel XV,Jaipur,2013,86999,First,23.08,1461,63.10,5,20.91683,3.50,1,0,0,1,11,1.5833486,54058.67
5,7,Toyota Innova Crysta 2.8 GX AT 8S,Mumbai,2016,36000,First,11.36,2755,171.50,8,21.00000,17.50,1,0,1,0,8,0.4375000,22369.36
6,8,Volkswagen Vento Diesel Comfortline,Pune,2013,64430,First,20.54,1598,103.60,5,20.91683,5.20,1,0,0,1,11,1.4288031,40034.95


## arrange

Here, we arrange the rows such that Price is sorted in ascending order (lowest prices first), and in case of a tie in the Price, Year will be used in descending order (newest cars first).

In [19]:
df %>% arrange(Price, desc(Year)) %>% head

,X,Name,Location,Year,Kilometers_Driven,Owner_Type,Mileage,Engine,Power,Seats,New_Price,Price,Fuel_TypeDiesel,Fuel_TypePetrol,TransmissionAutomatic,TransmissionManual,Age,Annual_Value_Loss
,<int>,<chr>,<chr>,<int>,<int>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,1713,Tata Nano Lx,Pune,2011,65000,Second,26.0,624,35,4,20.91683,0.44,0,1,0,1,13,1.5751411
2,2847,Hyundai Getz GLS,Pune,2005,86000,Second,15.3,1341,83,5,20.91683,0.45,0,1,0,1,19,1.0772018
3,1628,Maruti 800 Std BSIII,Jaipur,2004,12000,Second,16.1,796,37,4,20.91683,0.45,0,1,0,1,20,1.0233417
4,3138,Maruti Zen LXI,Jaipur,1998,95150,Third,17.3,993,60,5,20.91683,0.45,0,1,0,1,26,0.7871859
5,2605,Tata Nano Cx,Jaipur,2010,57000,First,26.0,624,35,4,20.91683,0.50,0,1,0,1,14,1.4583453
6,3228,Maruti 800 Std,Pune,2003,52000,First,16.1,796,37,4,20.91683,0.50,0,1,0,1,21,0.9722302


## summarize

This will summarize the data based on Location to show the number of rows, average price, median age, and number of diesel vehicles per location.

In [20]:
df %>%
    group_by(Location) %>%
    summarize(
        N = n(),
        Average_Price = mean(Price),
        Median_Age = median(Age),
        Num_Diesel = sum(Fuel_TypeDiesel)
    )

Location,N,Average_Price,Median_Age,Num_Diesel
<chr>,<int>,<dbl>,<dbl>,<dbl>
Ahmedabad,217,8.593134,10,137
Bangalore,347,13.638040,11,213
Chennai,473,7.968118,12,285
Coimbatore,629,15.140350,8,353
Delhi,535,9.905514,10,301
Hyderabad,707,10.014668,11,495
Jaipur,400,5.949225,11,227
Kochi,637,11.323940,8,303
Kolkata,521,5.739885,11,252
